In [1]:
import pandas as pd
import numpy as np
import copy
import itertools as itr
import wbdata
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

'''
%store -r wb_codes_df
%store -r world_1994
%store -r wb_data_df
%store -r world_wbdata_2016
%store -r fips_df
'''

pd.options.display.max_columns = 150
path = 'c:/Users/silas/MIDS/jupyternotebooks/USAID/'

In [2]:
wb_data_df = pd.read_pickle('{}wb_data_df.pickle'.format(path))
wb_codes_df = pd.read_pickle('{}wb_codes_df.pickle'.format(path))
fips_df= pd.read_pickle('{}fips_df.pickle'.format(path))
df_fa = pd.read_pickle('{}df_fa.pickle'.format(path))
fh_df = pd.read_pickle('{}fh_df.pickle'.format(path))

In [3]:
'''FA COLUMN HEADINGS
Index(['DateYear', 'country_id', 'country_code', 'country_name', 'region_id',
       'region_name', 'income_group_id', 'income_group_name',
       'income_group_acronym', 'implementing_agency_id',
       'implementing_agency_acronym', 'implementing_agency_name',
       'implementing_subagency_id', 'subagency_acronym', 'subagency_name',
       'channel_category_id', 'channel_category_name',
       'channel_subcategory_id', 'channel_subcategory_name', 'channel_id',
       'channel_name', 'dac_category_id', 'dac_category_name',
       'dac_sector_code', 'dac_sector_name', 'dac_purpose_code',
       'dac_purpose_name', 'funding_account_id', 'funding_account_name',
       'funding_agency_id', 'funding_agency_name', 'funding_agency_acronym',
       'assistance_category_id', 'assistance_category_name',
       'aid_type_group_id', 'aid_type_group_name', 'activity_id',
       'activity_name', 'activity_project_number', 'activity_start_date',
       'activity_end_date', 'transaction_type_id', 'transaction_type_name',
       'fiscal_year', 'current_amount', 'constant_amount', 'USG_sector_id',
       'USG_sector_name', 'framework', 'submission_id', 'numeric_year']
'''

'''Free (1.0 to 2.5), Partly Free (3.0 to 5.0), or Not Free (5.5 to 7.0)
https://freedomhouse.org/report/methodology-freedom-world-2017
'''

'Free (1.0 to 2.5), Partly Free (3.0 to 5.0), or Not Free (5.5 to 7.0)\nhttps://freedomhouse.org/report/methodology-freedom-world-2017\n'

### Freedom House dataframe preparation

In [4]:
# prepare for melt fh df so that every country has for each year its cl, pr, status
fh_df = fh_df.T.reset_index(drop=True)
fh_df.columns = fh_df.loc[0]
fh_df = fh_df.drop(0)
fh_df['Year(s) Under Review'] = fh_df['Year(s) Under Review'].ffill()

In [5]:
# corrects headers for FH dataset and prepares entries for merge with fa data
fh_c_list = list(fh_df.columns)
fh_c_list = [i.replace('&', 'and') for i in fh_c_list[2:]]
fh_c_list = pd.Series(fh_c_list).str.replace('Yugoslavia (Serbia and Montenegro)', 'Serbia and Montenegro')
fh_c_list = pd.Series(fh_c_list).str.replace(', The', '')
fh_c_list = list(fh_c_list)
fh_c_list.insert(0, 'FH_Score')
fh_c_list.insert(0, 'Year')
fh_df.columns = fh_c_list

In [6]:
# correcting year to single year note:a hack here where a single regex could work
fh_df['new'], fh_df['newer'] = fh_df['Year'].str.split('-').str
fh_df['new'] = fh_df['new'].str.replace(r'\D', '')
fh_df['Year'] = fh_df['new']
del fh_df['new']
del fh_df['newer']
# correct db for overlapping year
fh_df = fh_df.append(fh_df[27:30], ignore_index=True)

In [7]:
# create a new entry to get 1982 (because 1981 and 1982 were done in same year)
new_Y82 = fh_df.Year[132:]
fh_df.Year[132:] = list(map(lambda x: int(x)+1, new_Y82))

In [8]:
# bump each year up by one from '82 to '88
new_80s = fh_df.Year[30:51]
fh_df.Year[30:51] = list(map(lambda x: int(x)+1, new_80s))

In [9]:
fh_df.Year = fh_df.Year.astype(int)

In [10]:
fh_df = fh_df.sort_values(by='Year').reset_index(drop=True)

In [11]:
# switches columns FH score with Year
fh_melt = fh_df.set_index('FH_Score').reset_index()
# melts df on fh score and year
fh_melted = pd.melt(fh_melt, id_vars=['FH_Score', 'Year'], value_vars=list(fh_melt)[2:])
fh_melted.rename(columns={'variable':'country', 'value':'FH_value', 'Year': 'year'}, inplace=True)
# prep for selection
fh_melted.FH_Score = fh_melted['FH_Score'].str.strip()
fh_melted.FH_value = fh_melted.FH_value.str.replace(r"\(.*\)","")
fh_melted.FH_value = fh_melted.FH_value.replace('-',np.nan)
fh_melted.FH_value = fh_melted.FH_value.str.replace('PF ','PF')

In [12]:
# original shape (207, 133)
fh_melted.shape
# melted shape (27675, 4)  - 207*133 == 27675

(27675, 4)

### Foreign Aid dataframe preparation

In [13]:
# df_fa.country_name = pd.Series([d.get(e, e) for e in list(df_fa.country_name)])
# df_fa.country_name = df_fa.country_name.str.replace(r'\(.*', '')
# prepare FA df for merge with FH on year
# df_fa['index'] = df_fa.numeric_year.astype(int)
df_fa.rename(columns={'index': 'year','country_name': 'country' }, inplace=True) 
del df_fa['numeric_year']

In [14]:
# df_fa_prep = df_fa.groupby(['country_name', 'Year'])['current_amount', 'constant_amount'].mean()

# apply fips codes

In [15]:
# wb_codes = wbdata.api.get_country(country_id=None)

In [16]:
fips_df.head(1)
fh_melted.head(1)
df_fa.head(1)
wb_data_df.head(1)

,country,date,gdppc,gini,nat,pop,voice,year
0,Afghanistan,1960-01-01,NaN,NaN,NaN,8996351.0,NaN,1960


### a. helpers

In [17]:
def merge_dfs(dfl,dfr, i_on):
    merged_df = dfl.merge(dfr, how='left', on=i_on)
    print('DF shape: ', merged_df.shape)
    return merged_df

In [54]:
def pre_merge_dfs(df1_lst,df2_lst):
    print('number of countries in list L: ', len(df1_lst.country.unique()))
    print('number of years in list L: ', len(df1_lst.year.unique()))
    print('number of countries in list R: ', len(df2_lst.country.unique()))
    print('shape L: {}'.format(df1_lst.shape))
    print('shape R: {}'.format(df2_lst.shape))
    not_in = [x for x in df2_lst if x not in df1_lst]
    num_not = len(not_in)
    return print('{} L countries were not in merge:\n{} '.format(num_not, not_in))

### b. add fips to wb so that there are all fips countries on wb db

In [47]:
# wb_data_df has 273 unique countries because it includes lending groups such as IBRD
# years cover 1960 - 2017 = 58 years
# 273*53 = 14469

In [20]:
fips_df.rename(columns={'ISO3166_C':'wb_code'}, inplace=True)

In [22]:
# first add all WB codes to WB data df with merge
wb_w_wbcodes_df = merge_dfs(wb_data_df, wb_codes_df, 'country')

DF shape:  (15474, 9)


In [24]:
# second merge WB data df with fips df
wb_fipscoded = merge_dfs(wb_w_wbcodes_df, fips_df, 'wb_code')

DF shape:  (15532, 15)


In [25]:
del wb_fipscoded['country_y']
del wb_fipscoded['STANAG']

In [26]:
wb_fipscoded.rename(columns={'country_x':'country'}, inplace=True)

In [27]:
# no duplicate values
wb_fipscoded.drop_duplicates().shape == wb_fipscoded.shape

True

# fh to wb

In [28]:
chg_FH_WB = dict([('Bahamas','Bahamas, The'),
 ('Bosnia-Herzegovina','Bosnia and Herzegovina'),
 ('Brunei','Brunei Darussalam'),
 ('Cape Verde','Cabo Verde'),
 ('Congo (Brazzaville)', 'Congo, Rep.'),
 ('Congo (Kinshasa)', 'Congo, Dem. Rep.'),
 ('Egypt','Egypt, Arab Rep.'),
 ('Gambia','Gambia, The'),
 ('Iran', 'Iran, Islamic Rep.'),
 ('Kyrgyzstan','Kyrgyz Republic'),
 ('Laos','Lao PDR'),
 ('Macedonia','Macedonia, FYR'),
 ('Micronesia','Micronesia, Fed. Sts.'),
 ('North Korea','Korea, Dem. People���s Rep.'),
 ('Russia','Russian Federation'),
 ('Saint Kitts and Nevis','St. Kitts and Nevis'),
 ('Saint Lucia','St. Lucia'),
 ('Saint Vincent and Grenadines','St. Vincent and the Grenadines'),
 ('Slovakia','Slovak Republic'),
 ('South Korea','Korea, Rep.'),
 ('Syria', 'Syrian Arab Republic'),
 ('Taiwan','Taiwan, China'),
 ('Venezuela', 'Venezuela, RB'),
 ('Yemen','Yemen, Rep.')])

In [57]:
pre_merge_dfs(wb_fipscoded, fh_melted)

number of countries in list L:  273
number of years in list L:  58
number of countries in list R:  205
shape L: (15532, 13)
shape R: (27675, 4)
2 L countries were not in merge:
['FH_Score', 'FH_value'] 


In [30]:
# pre_merge_dfs(fh_melted.country.unique(), wb_fipscoded.country.unique())

In [31]:
# change non-fit country names here
fh_melted.country = pd.Series([chg_FH_WB.get(e, e) for e in list(fh_melted.country)])

In [32]:
# l merge wb into fh
fh_w_wb_df = merge_dfs(fh_melted, wb_fipscoded,  ['country', 'year'])

DF shape:  (27675, 15)


In [33]:
fh_to_add_to_wb = ['Czechoslovakia',
 'Germany, E. ',
 'Germany, W. ',
 'USSR',
 'Vietnam, N.',
 'Vietnam, S.',
 'Yemen, N.',
 'Yemen, S.',
 'Yugoslavia',
 'Yugoslavia (Serbia and Montenegro)']

In [34]:
fh_w_wb_df.drop_duplicates().shape == fh_w_wb_df.shape

True

In [35]:
with open( "{}fh_w_wb_df.pickle".format(path), "wb" ) as f:
    pickle.dump(fh_w_wb_df, f)
# pickle.load( open( "{}fh_w_wb_df.pickle".format(path), "rb" ) )

# FHWB to FA

In [36]:
df_fa.rename(columns={'country_code':'wb_code', 'fiscal_year':'year'}, inplace=True)

In [37]:
df_fa.year = df_fa.year.replace('1976tq','1976').astype(int)

In [38]:
fhfawb = fh_w_wb_df.merge(df_fa, how='left', on=['wb_code', 'year'])

In [39]:
fhfawb.rename(columns={'country_x':'country'}, inplace=True)

In [40]:
fhfawb.to_pickle("{}fhfawb.pickle".format(path))

In [41]:
# no wb_country code
fh_w_wb_df[fh_w_wb_df.wb_code.isnull()==True].country.unique()

array(['Czechoslovakia', 'Germany, E. ', 'Germany, W. ',
       'Korea, Dem. People���s Rep.', 'Taiwan, China', 'USSR',
       'Vietnam, N.', 'Vietnam, S.', 'Yemen, N.', 'Yemen, S.',
       'Yugoslavia', 'Yugoslavia (Serbia and Montenegro)'], dtype=object)